#### OBJECTIVES

1. pull information going back to 2000
2. breakdown port allocations in key reusable methods
3. implement date rules to perform optimization at given intervals (date resample)
4. calculate cumulative portfolio returns as portfolio allocation changes
5. run sensitivities fine-tunning opt parameters (MVO day window, pos sizing, leverage)

#### Helper Methods

In [ ]:
from math import *
from datetime import datetime, date, time, timedelta
import pandas_datareader.data as web
import numpy as np
import pandas as pd
import cvxpy as cvx
import re, os
import matplotlib.pyplot as plt 

pattern = r'holdings-'
path = "./sector_components/"
date_fmt = '%m-%d-%Y'
log = False

ticker_map = {
    'benchmark': ['SPY'],
    'equity': ['VTI','VTV','VOE','VBR','VEA','VWO'],
    'fixed_income': ['VTIP', 'SHV', 'MUB', 'LQD', 'BNDX', 'EMB'],
    'spy_sectors': ['XLE', 'XLU', 'XLK', 'XLB', 'XLP', 'XLY', 'XLI', 'XLV', 'XLF', 'XLRE']
}

dwld_key = 'XLV'
sectors = ticker_map['spy_sectors']
sector_tickers_map = {}

%matplotlib inline
%config InlineBackend.figure_format = 'svg'

In [ ]:
key = list(ticker_map.keys())[3]
print("retrieving prices for:", key, ticker_map[key])

In [ ]:
compound = lambda x: (x + 1).cumprod()
two_dec = lambda x: '%.4f' % x
# need to fix this method
def get_pricing(fname, ticker_list, start_date):
    if log: print("Getting pricing for:", ticker_list, start_date)
    px = web.DataReader(ticker_list,data_source='yahoo',start=start_date)['Adj Close']
    px.to_csv(fname)
    return px
def show_weights(weights, labels, ret, sigma):
    df = pd.DataFrame(weights, columns=labels)
    df['return'] = ret * 252
    df['sigma'] = sigma * np.sqrt(252)
    df['sharpe'] = df['return'] / df['sigma']
    return df

In [ ]:
def get_mean_variance(rets):
    w_len = rets.shape[1] # number of columns
    eq_weights = np.asarray([1/w_len for _ in range(w_len)]) #default weights
    mu = rets.mean()
    std_dev = rets.std()
    cov_matrix = rets.cov()
    return w_len, eq_weights, mu.values, std_dev, cov_matrix.values    
#this replaces the method above WIP
def get_mvo_allocations(rets, min_sum=1, max_sum=1, min_w=0, max_w=0.2):

    w_len = rets.shape[1] # number of columns
    eq_weights = np.asarray([1/w_len for _ in range(w_len)]) #default weights    
    mu, Sigma, w = rets.mean().T, rets.cov(), cvx.Variable(w_len)
    
    gamma = cvx.Parameter(sign='positive')
    ret = mu.T * w 
    risk = cvx.quad_form(w, Sigma)
    prob = cvx.Problem(cvx.Maximize(ret - gamma*risk), 
        [cvx.sum_entries(w) >= min_sum, 
         cvx.sum_entries(w) <= max_sum, 
         w > min_w,
         w < max_w])
    gamma.value = 0.5; prob.solve()
    if prob.status == 'optimal': return [i[0] for i in w.value.tolist()]
def get_mvo_allocations(n, mu_ret, cov_mtrx, min_sum=1, max_sum=1, min_w=0, max_w=0.2):
    mu = mu_ret.T
    Sigma = cov_mtrx
    w = cvx.Variable(n)
    gamma = cvx.Parameter(sign='positive')
    ret = mu.T * w 
    risk = cvx.quad_form(w, Sigma)
    prob = cvx.Problem(cvx.Maximize(ret - gamma*risk), 
        [cvx.sum_entries(w) >= min_sum, 
         cvx.sum_entries(w) <= max_sum, 
         w > min_w,
         w < max_w])
    gamma.value = 0.5; prob.solve()
    if prob.status == 'optimal': 
        return [i[0] for i in w.value.tolist()]

In [ ]:
p_template = "Ann. return: {0:.2f}%, std dev: {1:.2f}%, sharpe: {2:.2f}"
def calc_port_performance(arr, weights):
    return np.cumprod(np.sum(arr * weights, axis=1) + 1)
def date_rules(date_range, tgt_date_str, freq):
    #return a list of dates
    tgt_dt = datetime.strptime(tgt_date_str, date_fmt)
    return date_range[:date_range.index(tgt_dt)+1][::-freq]
def date_intervals(df, freq):
    #using pandas
    return df.resample(freq, closed='left', label='left').mean()
def portfolio_metrics(pdf):
    ret = (pdf.pct_change().mean() * 252).values[0]
    std = (pdf.pct_change().std() * sqrt(252)).values[0]
    print(p_template.format(ret * 100, std * 100, ret / std))
    return ret, std, ret / std
def recomend_allocs(w, irange, top):
    w = (w.loc[irange].sum(axis=0).sort_values(ascending=False) / max_w_const).astype(np.int)
    return w[:top]

In [ ]:
def get_weights(px, freq, lb=20, min_sum=1, max_sum=1, min_w=0, max_w=0.1):
    px.dropna(axis=1, inplace=True)
    returns = px.sort_index().pct_change(); returns.iloc[0] = 0
    intervals = pd.to_datetime(date_intervals(returns, freq).index.tolist())
    valid_dates = [d for d in intervals if d in returns.index]    
    #cols = returns.columns    
    hist_alloc = pd.DataFrame(np.zeros((returns.shape)), index=returns.index, columns=returns.columns)
    if log: 
        print("Empty allocations:", hist_alloc.shape)
        print('{0:d} stocks, {1:d} days, {2:d} lookback'.format(len(returns.columns), len(px), lb))

    for i in valid_dates:
        lb_returns = returns.loc[:i.date()].tail(lb).dropna()
        weights = np.array([0 for _ in range(len(returns.columns))])
        if (len(lb_returns) > 2):
            n, weights, mu_ret, std_dev, cov_mtrx = get_mean_variance(lb_returns)
            weights = get_mvo_allocations(
                n, mu_ret, cov_mtrx, min_sum=min_sum, max_sum=max_sum, min_w=min_w, max_w=max_w)
        hist_alloc.loc[i.date()] = weights

    hist_alloc = hist_alloc.loc[returns.index].replace(0, np.nan).fillna(method='ffill')
    hist_alloc.replace(np.nan, 0, inplace=True)
    
    if log: print("returns: rows / cols", returns.shape, "allocation: rows / cols", hist_alloc.shape)
    return returns, hist_alloc

#### Test Methods

In [ ]:
np.random.seed(42)
numdays, cols = 100, 10
end_date_str, tgt_date_str = '12-31-2017', '12-27-2017'
freq = 7; lookback = 20

arr = (np.random.rand(numdays, cols) - 0.5) / 10
weights = np.random.rand(1, cols)
weights = weights / np.sum(weights, axis=1).T

In [ ]:
#test the portfolio performance calculation
port_perf = calc_port_performance(arr, weights)
#pd.DataFrame(port_perf).plot()
port_perf

In [ ]:
#test the date rules / intervals
end_date = datetime.strptime(end_date_str, date_fmt)
d_rng = sorted([end_date - timedelta(x) for x in range(0, numdays)]) # using list comprenhensions
sorted(date_rules(d_rng, tgt_date_str, freq))

d_rng = pd.date_range(end=end_date_str, freq='D', periods=numdays) # using pandas date range
d_rng = list(pd.to_datetime(d_rng))
intervals = list(sorted(date_rules(d_rng, tgt_date_str, freq)))
print("check:", len(intervals), "equals", numdays // freq, "result:",len(intervals) == numdays // freq) # check if intervals works
intervals[-5:]

In [ ]:
d_rng = pd.date_range(end=end_date_str, freq='D', periods=numdays) # using pandas date range
d_rng = list(pd.to_datetime(d_rng))

df = pd.DataFrame(arr, index=d_rng, columns=[i for i in range(cols)])
(df+1).cumprod().mean(axis=1).plot()

In [ ]:
#test both the portfolio performance using date intervals without optimization / equal weights
date_range = list(df.index)
intervals = list(sorted(date_rules(date_range, tgt_date_str, freq)))
hist_alloc = pd.DataFrame(np.zeros((len(df),cols)), index=df.index)

for i in intervals:
    #lb_returns = df.loc[:i.date()].tail(lookback)
    weights = np.array([1/cols for _ in range(cols)])
    #print(['{0:.2f}'.format(x) for x in weights])
    hist_alloc.loc[i.date()] = weights

hist_alloc.loc[intervals[0]:] = hist_alloc.loc[intervals[0]:].replace(0, np.nan).fillna(method='ffill')

port_perf = calc_port_performance(df.values, hist_alloc.values)
pd.DataFrame(port_perf).plot()
port_perf[-1:]

In [ ]:
#test both the portfolio performance using date intervals with optimization
date_range = list(df.index)
intervals = list(sorted(date_rules(date_range, tgt_date_str, freq)))
hist_alloc = pd.DataFrame(np.zeros((len(df),cols)), index=df.index)

for i in intervals:
    lb_returns = df.loc[:i.date()].tail(lookback)
    n, weights, mean_returns, std_dev, cov_matrix = get_mean_variance(lb_returns)
    weights = get_mvo_allocations(n, mean_returns, cov_matrix, min_w=0.0, max_w=0.3)
    #print(['{0:.2f}'.format(x) for x in weights])
    hist_alloc.loc[i.date()] = weights

hist_alloc.loc[intervals[0]:] = hist_alloc.loc[intervals[0]:].replace(0, np.nan).fillna(method='ffill')
hist_alloc

port_perf = calc_port_performance(df.values, hist_alloc.values)
pdf = pd.DataFrame(port_perf)
pdf.plot()
port_perf[-1:]

#### Load from hard-drive

In [ ]:
# load sector components
flist = os.listdir(path)
files = [f for f in flist if f.startswith(pattern)]
colstoload = ['Symbol','Company Name', 'Index Weight']
numdays, cols = 252, 10; freq = "W-WED"; lookback = 20; hist_window = 252*5
end_date_str = tgt_date_str = '1-8-2018'
start_date = datetime.strptime('1-8-2018', date_fmt)
start_date = start_date - timedelta(hist_window)
companies = pd.DataFrame([])

for s in sectors:
    fname = path + pattern + s.lower() + '.csv'
    df = pd.read_csv(fname, skiprows=1, index_col='Symbol', usecols=colstoload)
    df['ETF'] = s
    sector_tickers_map[s] = df.index.tolist()
    companies = companies.append(df)

if log: print("Company Sample:", companies.shape); print(companies.groupby('ETF')['Index Weight'].count())

# LOAD FROM HARD DRIVE
px = pd.read_csv(dwld_key + '-hold-pricing.csv', index_col='Date', parse_dates=True)
spyder_etf = pd.read_csv(dwld_key + '.csv', index_col='Date', parse_dates=True)

#### Get Data from the Server

In [ ]:
# HITS THE SERVER: downloads data from yahoo for all tickers for a given sector + ETF for same date range
tickers = sector_tickers_map[dwld_key] # for individual components
#tickers = ticker_map["spy_sectors"] # for individual ETFs
px = get_pricing(dwld_key + '-hold-pricing.csv', tickers, start_date.strftime(date_fmt))
etf = get_pricing(dwld_key + '.csv', dwld_key, start_date.strftime(date_fmt))
spyder_etf = pd.DataFrame(etf)
spyder_etf.index.name = "Date"
spyder_etf.columns=[dwld_key]
spyder_etf.to_csv(dwld_key + '.csv')

In [ ]:
frame = (-252*3); max_w_const = 0.1
px.dropna(axis=0, inplace=True)
px.dropna(axis=1, inplace=True)
px_portion = px[frame:].copy()
s_etf = (spyder_etf[frame:].pct_change() + 1).cumprod()
returns, alloc = get_weights(px_portion, "W-WED", max_w=max_w_const)
port_perf = calc_port_performance(returns.values, alloc.values)
pdf = pd.DataFrame(port_perf, index=returns.index, columns=[dwld_key + "-cvxopt"])
portfolio_metrics(pdf);

In [ ]:
ax = pdf.plot(); s_etf.plot(ax=ax, legend='right')

In [ ]:
msg = "Portfolio metrics starting every {} trading days and holding for {} days"
holding = 252; stop = int(len(alloc) - holding); jumps = 14
offsets = [x for x in range(0, stop, jumps)]
print(msg.format(jumps, holding))
results = []
for o in offsets:
    start = np.min([o, len(alloc)-1])
    end = np.min([o+holding, len(alloc)])
    p = px[start:end].copy()
    s_etf = (spyder_etf[start:end].pct_change() + 1).cumprod()
    r, w = get_weights(p, "W-WED", max_w=0.10)
    port_perf = calc_port_performance(r.values, w.values)
    pdf = pd.DataFrame(port_perf, index=r.index) # index by date
    results.append(pdf[-1:].values[0][0])
    #portfolio_metrics(pdf)
pd.DataFrame(results, columns=["Return"]).plot()

In [ ]:
# show portfolio metrics for a given time window
length = 252
w = alloc[-length:].astype(np.float)
r = returns[-length:].astype(np.float)
port_perf = calc_port_performance(r.values, w.values)
pdf = pd.DataFrame(port_perf, index=r.index, columns=[dwld_key + "-cvxopt"])
portfolio_metrics(pdf);

In [ ]:
# show top holdings and last recomended holding set
w = alloc[-length:].astype(np.float16)
intervals = pd.to_datetime(date_intervals(r, freq).index.tolist())

top = 10
irange = intervals
print("Top {} holdings during the last {} intervals".format(top, len(irange)))
print(recomend_allocs(w, irange, top))

irange = intervals[-5:]
print("Top {} holdings during the last {} intervals".format(top, len(irange)))
print(recomend_allocs(w, irange, top))

irange = intervals[-1:]
print("Top {} holdings during the last {} intervals".format(top, len(irange)))
print(recomend_allocs(w, irange, top))

In [ ]:
# show index return plots by year
first_year = int(alloc.index[0].year)
last_year = int(alloc.index[-1].year)
years = [y for y in range(first_year, last_year, 1)] 

def perf_by_years(r, a, years):
    ax = plt.axes()
    for y in years:
        year = str(y)
        w = alloc.loc[year].astype(np.float16)
        r = returns.loc[year].astype(np.float16)
        p_perf = calc_port_performance(r.values, w.values)
        result = pd.DataFrame(p_perf, index=w.index, columns=[year])
        result.plot(title='Optimal Components of ' + dwld_key, ax=ax, legend='right')
        #print(year, result[-1:].values[0][0])

perf_by_years(returns, alloc, years)

In [ ]:
#CHECK compounding math
#what were the top 10 allocations tickers?
top_stocks = alloc.sum(axis=0).sort_values(ascending=False)[:10].index.tolist()
# what was their allocation?
alloc = alloc[top_stocks]
# how much did we allocate to them?
cum_alloc = alloc.sum(axis=1)
# multiply the daily returns of top allocations times our allocation
port_return = (returns[top_stocks] * alloc).sum(axis=1)
# we add 1 to get the compounding index
port_index = (port_return + 1).cumprod()
#cumulative return for the portfolio
print(port_index[-1:], len(port_index), "days")

port_perf = calc_port_performance(returns[top_stocks].values, alloc.values)
print(port_perf[-1:], len(port_perf), "days")
print("annual return", pd.Series(port_perf).pct_change().mean() * 252)

In [ ]:
# show behaviour during sepcific time window
window = pdf.loc['2015-1-1':'2015-3-31']
portfolio_metrics(window)
window.plot();

#### Sensitivities

In [565]:
lbs = [x for x in range(5, 15, 5)]
mws = (np.array([x for x in range(4, 24, 4)]) / 100).tolist()
for i, l in enumerate(lbs):
    for j, w in enumerate(mws):
        print(l, w)

5 0.04
5 0.08
5 0.12
5 0.16
5 0.2
10 0.04
10 0.08
10 0.12
10 0.16
10 0.2


In [580]:
def create_matrix(px, start, end, step):
    lbs = [x for x in range(start, end, step)]
    mws = (np.array([x for x in range(start, end, step)]) / 100).tolist()
    df = pd.DataFrame([], index=mws, columns=lbs)
    
    for i, l in enumerate(lbs):
        for j, w in enumerate(mws):
            r, w = get_weights(px_portion, freq, lb=l, max_w=w)
            port_perf = calc_port_performance(r.values, w.values)
            pdf = pd.DataFrame(port_perf, index=r.index, columns=[dwld_key + "-cvxopt"])
            days = len(pdf)
            ret, std, sharpe = portfolio_metrics(pdf);
            df.iloc[i, j] = (
                ret.astype(np.float16), 
                std.astype(np.float16), 
                sharpe.astype(np.float16))
    return df

def heatmap(df, cmap = plt.cm.gray_r): 
    fig = plt.figure() 
    ax = fig.add_subplot(111) 
    axim = ax.imshow(df.values, cmap=cmap, interpolation='nearest')
    ax.set_xlabel(df.columns.name) 
    ax.set_xticks(np.arange(len(df.columns)))
    ax.set_xticklabels(list(df.columns))
    ax.set_ylabel(df.index.name)
    ax.set_yticks(np.arange(len(df.index)))
    ax.set_yticklabels( list(df.index))
    plt.colorbar(axim)

In [582]:
sm = create_matrix(px, 4, 24, 4)
#heatmap(st)

Ann. return: 41.35%, std dev: 16.41%, sharpe: 2.52
Ann. return: 55.33%, std dev: 18.86%, sharpe: 2.93
Ann. return: 66.43%, std dev: 20.65%, sharpe: 3.22
Ann. return: 72.59%, std dev: 22.19%, sharpe: 3.27
Ann. return: 33.41%, std dev: 16.11%, sharpe: 2.07
Ann. return: 45.09%, std dev: 18.11%, sharpe: 2.49
Ann. return: 47.83%, std dev: 19.63%, sharpe: 2.44
Ann. return: 52.07%, std dev: 21.36%, sharpe: 2.44
Ann. return: 30.57%, std dev: 15.71%, sharpe: 1.95
Ann. return: 37.19%, std dev: 17.51%, sharpe: 2.12
Ann. return: 41.23%, std dev: 19.00%, sharpe: 2.17
Ann. return: 45.14%, std dev: 20.74%, sharpe: 2.18
Ann. return: 27.87%, std dev: 15.68%, sharpe: 1.78
Ann. return: 34.82%, std dev: 17.42%, sharpe: 2.00
Ann. return: 38.29%, std dev: 19.07%, sharpe: 2.01
Ann. return: 40.10%, std dev: 20.51%, sharpe: 1.95


In [597]:
extract = lambda x: x[2]
sm.applymap(extract)

,4,8,12,16
0.04,2.519531,2.933594,3.216797,3.271484
0.08,2.074219,2.490234,2.435547,2.437500
0.12,1.946289,2.125000,2.169922,2.175781
0.16,1.777344,1.998047,2.007812,1.955078


#### Old Scripts

In [ ]:
for s in sector_tickers_map.keys():
    print(len(sector_tickers_map[s]))
    
#test both the portfolio performance using date intervals with optimization
df = pd.DataFrame(arr, index=d_rng, columns=[i for i in range(cols)])
date_range = list(df.index)
intervals = list(sorted(date_rules(date_range, tgt_date_str, freq)))
hist_allocations = pd.DataFrame(np.zeros((len(intervals),cols)), index=pd.to_datetime(intervals))

for i in intervals:
    lb_returns = df.loc[:i.date()].tail(lookback)
    w_len, weights, mean_returns, std_dev, cov_matrix = get_mean_variance(lb_returns)
    weights = get_mvo_allocations(mean_returns, cov_matrix)
    hist_allocations.loc[i.date()] = weights

port_perf = calc_port_performance(df.loc[intervals].values, hist_allocations.values)
pd.DataFrame(port_perf).plot()
port_perf[-1:]

In [ ]:
px = get_pricing(ticker_map[key], '01/01/2017')
returns = px.sort_index().pct_change()
compound(returns).plot()

In [ ]:
w_len, weights, mean_returns, std_dev, cov_matrix = get_mean_variance(returns)

ann_returns = np.dot((mean_returns * 252), weights)
ann_stdev = np.sqrt(252/len(returns)) * std_dev
print(weights.shape, cov_matrix.shape)
port_variance = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights))) * np.sqrt(252)
print("eq weight return(exp)", ann_returns)
print("port risk(exp):", port_variance)
print("sharpe ratio:", ann_returns / port_variance)

In [ ]:
# Long only portfolio optimization.
weights = get_mvo_allocations(mean_returns, cov_matrix)
np_weights = np.array([weights]).T
exp_return = np.dot(np.array([mean_daily_returns.values]), np_weights) * 252
portfolio_std_dev = np.sqrt(np.dot(np_weights.T, np.dot(cov_matrix, np_weights))) * np.sqrt(252)
print("optimized return(exp):", exp_return)
print("optimized portfolio risk(exp):", portfolio_std_dev)
print("sharpe ratio:", exp_return / portfolio_std_dev)

In [ ]:
# Compute trade-off curve.
SAMPLES = 100
weights = []
risk_data = np.zeros(SAMPLES)
ret_data = np.zeros(SAMPLES)
gamma_vals = np.logspace(-2, 3, num=SAMPLES)
for i in range(SAMPLES):
    gamma.value = gamma_vals[i]
    prob.solve()
    weights.append([i[0] for i in w.value.tolist()])
    risk_data[i] = cvx.sqrt(risk).value
    ret_data[i] = ret.value
print('Optimization status:', prob.status)
#w.value, risk_data, ret_data
#ret_data / risk_data # sharpe ratio
#risk_data[np.argmin(risk_data)], risk_data[np.argmax(ret_data)]
#wgt_cum_ret = (ret_data + 1).cumprod()
cols = returns.columns.tolist();
allocs = show_weights(weights, returns.columns, ret_data, risk_data); allocs.tail()
allocs[cols].plot()
print(allocs[-1:].apply(two_dec))

In [ ]:
# Plot long only trade-off curve.
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

markers_on = range(1, 100, 10)
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(risk_data, ret_data, 'g-')
for marker in markers_on:
    plt.plot(risk_data[marker], ret_data[marker], 'bs')
    #ax.annotate(r"$\gamma = %.2f$" % gamma_vals[marker], xy=(risk_data[marker], ret_data[marker]))
for i in range(n):
    plt.plot(sqrt(Sigma[i,i]).value, mu[i], 'ro')
    ax.annotate(returns.columns[i], xy=(sqrt(Sigma[i,i]).value, mu[i]))
plt.xlabel('Standard deviation')
plt.ylabel('Return')
plt.show()

In [ ]:
gamma_vals.shape, risk_data.shape, ret_data.shape
summary = pd.DataFrame([], columns=['gamma', 'risk', 'return'], index=range(SAMPLES))
summary['gamma'] = np.array([gamma_vals]).T
summary['risk'] = np.array([risk_data]).T
summary['return'] = np.array([ret_data]).T
summary[['risk','return']].plot(kind='line')